## Test Emotion for Maya

In [6]:
import socket
import numpy as np
import pandas as pd
from DataAPI import MayaController
import json
import os
import copy
from IPython.display import clear_output
from csv import writer

#### Open server

import maya.cmds as cmds

cmds.commandPort(n="localhost:12345")

cmds.commandPort(cl=True, n="localhost:12345")

In [8]:
controller = MayaController()

### Data Interpolation

In [31]:
df = pd.read_csv("../data/emotions.csv")
df

,Name,Brow_L_X,Brow_L_Y,Brow_R_X,Brow_R_Y,Mouth_M_X,Mouth_M_Y,Mouth_M_Lower,Mouth_M_Upper,Mouth_L_X,Mouth_L_Y,Mouth_R_X,Mouth_R_Y,Eye_L_S,Eue_R_S,V,A,D
0,Scared,0.500000,0.1,0.500000,0.1,0.2,-0.700000,0,0,2.780000e-17,0.0,2.780000e-17,0.0,0,0,0.146,0.828,0.185
1,Angry,-0.600000,-0.4,-0.600000,-0.4,0.0,0.000000,10,10,2.780000e-17,-0.2,2.780000e-17,-0.2,7,7,0.122,0.830,0.604
2,Curious,0.300000,0.0,0.300000,0.0,0.0,-0.300000,0,0,2.780000e-17,0.2,2.780000e-17,0.2,0,0,0.635,0.600,0.483
3,Happy,0.025824,-0.2,0.025824,-0.2,0.4,-0.160208,0,0,3.000000e-01,1.0,3.000000e-01,1.0,0,0,1.000,0.735,0.772
4,Neutral,0.000000,0.0,0.000000,0.0,0.0,0.000000,0,0,0.000000e+00,0.0,0.000000e+00,0.0,0,0,0.469,0.184,0.357
5,Sad,0.500000,-0.4,0.500000,-0.4,0.0,-0.075341,0,0,1.000000e-01,-1.0,1.000000e-01,-1.0,5,5,0.225,0.335,0.149
6,Concerned,0.000000,-0.1,0.000000,-0.1,0.0,0.000000,5,5,-2.000000e-01,-0.2,2.000000e-01,-0.6,2,2,0.292,0.698,0.350
7,Confused,-0.500000,-0.1,-0.300000,-0.1,0.1,0.000000,10,0,2.780000e-17,-0.1,4.000000e-01,-0.4,3,4,0.220,0.650,0.179
8,Shy,0.000000,-0.1,0.300000,0.0,0.2,0.000000,0,0,2.780000e-17,0.6,-1.000000e-01,0.2,0,0,0.280,0.153,0.132
9,Frustrated,0.000000,-0.3,0.000000,-0.3,0.0,0.000000,5,5,-2.000000e-01,0.0,2.000000e-01,-0.4,2,2,0.080,0.651,0.255


In [27]:
def print_json(j):
    print(json.dumps(j, indent=4))

def StoreDataToCSV(data: dict, file_path: str, vad:list, name:str="N/A"):
    '''
    Save the skeleton data to csv
    '''
    emo_lst = [name]
    for attr_dict in data.values():
        for attr in attr_dict.values():
            emo_lst.append(attr)
    emo_lst += vad

    with open(file_path, 'a') as write_obj:
        csv_writer = writer(write_obj)
        csv_writer.writerow(emo_lst)


In [43]:
def EmotionInterpolation(load_dir: str, save_dir: str, n:int =1):
    '''
    Interpolate the emotions and save both the screenshots and the facial attributes as dictionaries
    :param
        load_dir: loading directory of the facial poses
        save_dir: saving directory of the generated screenshots
        n: number of interpolations between poses 
    :return: list of interpolated emotion attributes
    '''
    emo_lst = {}
    # Get every poses
    poses_dir = os.listdir(load_dir)

    # for strt_pose, end_pose in itertools.combinations(poses_dir, r=2):
    strt_pose = poses_dir[0]
    end_pose = poses_dir[1]
    for i in range(1):
        # Get the facial attributes of the starting and ending poses
        strt_dict = controller.GetFacialAttributesFromPose(os.path.join(os.path.join(load_dir, strt_pose), "pose.json"))
        end_dict = controller.GetFacialAttributesFromPose(os.path.join(os.path.join(load_dir, end_pose), "pose.json"))

        for joint, attr_dict in strt_dict.items():
            for name, value in attr_dict.items():
                diff = (end_dict[joint][name] - value) / (n + 1)
                if diff:
                    # Interpolate new emotions
                    for i in range(n + 2):
                        new_dict = copy.deepcopy(strt_dict)
                        new_dict[joint][name] += i * diff
                        index_str = strt_pose.split('.')[0] + "-" + end_pose.split('.')[0] + "." + joint + "." + name + "." + str(i)
                        emo_lst[index_str] = new_dict
                        # Apply Emotion and save a screenshot
                        controller.SetFacialAttributes(new_dict)
                        controller.ScreenShot(os.path.join(save_dir, index_str))
                        clear_output(wait=True)

    return emo_lst

In [44]:
emo_dict = EmotionInterpolation("../StudioLibrary/Poses/Face", "/home/vince/Documents/research/other/img")

36


In [11]:
print("Number of data generated: %f\n" %len(emo_dict))
print_json(emo_dict)


Number of data generated: 1540.000000

{
    "Scared-Angry.ctrlBrow_L.translateX.0": {
        "ctrlBrow_L": {
            "translateX": -0.050000000000000044,
            "translateY": -0.15000000000000357
        },
        "ctrlBrow_R": {
            "translateX": -0.050000000000000044,
            "translateY": -0.15000000000000357
        },
        "ctrlMouth_M": {
            "translateX": 0.1,
            "translateY": -0.35,
            "lowerSqueeze": 5.0,
            "upperSqueeze": 5.0
        },
        "ctrlMouthCorner_L": {
            "translateX": 2.7755575615628914e-17,
            "translateY": -0.09999999999999999
        },
        "ctrlMouthCorner_R": {
            "translateX": 2.7755575615628914e-17,
            "translateY": -0.1
        },
        "ctrlEye_L": {
            "squint": 3.5
        },
        "ctrlEye_R": {
            "squint": 3.5
        }
    },
    "Scared-Angry.ctrlBrow_L.translateX.1": {
        "ctrlBrow_L": {
            "translateX": -0

In [ ]:
controller.close()